In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count, coalesce
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead, row_number, min,max, first
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
import time

# from mock.tasks import adiciona_carro}
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder.appName("test").getOrCreate()
sql = SQLContext(ss)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

session.execute("USE simulacao")

import json
params = json.load(open('./mock/parametros.json'))

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
fps = 5
p = [[key]+list(params[key].values()) for key in params.keys()]
p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))
df_old = ss.createDataFrame([], "placa: string, posicao: int, faixa: int, rodovia: string, tempo_da_simulacao: int, velocidade: double, aceleracao: double, posicao_prevista: double, acima_vel: boolean, aplicaMulta: boolean, tempo_em_curso: int,tempo_inicio: int")
rodovias = p.select(F.collect_list('rodovia')).collect()[0][0]
vel_media = {rodovia:0 for rodovia in rodovias} #"BR-116": 0, "BR-040": 0, "BR-135": 0, "BR-393": 0}
n_vel_media = {rodovia:0 for rodovia in rodovias} # {"BR-116": 0, "BR-040": 0, "BR-135": 0, "BR-393": 0}

tempo_medio = {rodovia:0 for rodovia in rodovias}
n_tempo_medio = {rodovia:0 for rodovia in rodovias}

def atualiza_media(media_atual, tamanho_atual, media_add, tamanho_add):
    if media_add == None:
        return media_atual
    if tamanho_atual == 0:
        return media_add
    tamanho_total = tamanho_atual + tamanho_add
    return (media_atual/tamanho_total)*tamanho_atual + (media_add/tamanho_total)*tamanho_add

def processa_velocidade_media(batch):
    global vel_media, n_vel_media
    batch = batch.na.fill(0, subset=['vel_y'])
    
    # group by "rodovia" and aggregate the mean of "velocidade"
    mean_df = batch.groupBy("rodovia").agg(F.mean(F.abs("vel_y").alias('vel_y')).alias('vel_y'))
    # collect the rows as a list
    mean_rows = mean_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    mean_dict = {row.asDict()["rodovia"]: row.asDict()["vel_y"] for row in mean_rows}
    # group by "rodovia" and aggregate the mean of "velocidade"
    length_df = batch.groupBy("rodovia").agg(F.count("vel_y"))
    length_df = length_df.withColumnRenamed("count(vel_y)", "vel_y")
    # collect the rows as a list
    length_rows = length_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    length_dict = {row.asDict()["rodovia"]: row.asDict()["vel_y"] for row in length_rows}
    for key in length_dict.keys():
        n_vel_media[key]+=length_dict[key]
        vel_media[key] = atualiza_media(vel_media[key], n_vel_media[key], mean_dict[key], length_dict[key])
    pass

def processa_tempo_cruzamento(batch):
    global n_tempo_medio, tempo_medio
    batch = batch.filter(F.col("tempo_cruzamento").isNotNull())
    # group by "rodovia" and aggregate the mean of "velocidade"
    mean_df = batch.groupBy("rodovia").agg(F.mean("tempo_cruzamento"))
    mean_df = mean_df.withColumnRenamed("avg(tempo_cruzamento)", "tempo_cruzamento")
    # collect the rows as a list
    mean_rows = mean_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    mean_dict = {row.asDict()["rodovia"]: row.asDict()["tempo_cruzamento"] for row in mean_rows}

    # group by "rodovia" and aggregate the mean of "velocidade"
    length_df = batch.groupBy("rodovia").agg(F.count("tempo_cruzamento"))
    length_df = length_df.withColumnRenamed("count(tempo_cruzamento)", "tempo_cruzamento")
    # collect the rows as a list
    length_rows = length_df.collect()
    # create a dictionary with "rodovia" as key and mean as value
    length_dict = {row.asDict()["rodovia"]: row.asDict()["tempo_cruzamento"] for row in length_rows}
    n_tempo_medio = {key: n_tempo_medio[key]+length_dict[key] for key in list(n_tempo_medio.keys())}
    tempo_medio = {key: atualiza_media(tempo_medio[key], n_tempo_medio[key], mean_dict[key], length_dict[key]) for key in list(tempo_medio.keys())}
    pass

In [34]:
a, b = 0, 100000
query = f"SELECT * FROM simulacao WHERE tempo_da_simulacao >= {a} AND tempo_da_simulacao <= {b} ALLOW FILTERING;"
r = list(session.execute(query))
if r != []:
    df = ss.createDataFrame(r)

In [35]:
df.count()

11121

In [36]:
df.show()

+-------+------------------+-------+-----+-------------------+
|rodovia|tempo_da_simulacao|  placa|pos_x|              pos_y|
+-------+------------------+-------+-----+-------------------+
| BR-040|                48|ARG3R45|655.0|  629.7999999999995|
| BR-040|                48|BOL7C89|385.0|             -131.9|
| BR-040|                48|BRA6M78|475.0|-113.41999999999999|
| BR-040|                48|CHI0U12|565.0|-25.300000000000026|
| BR-040|                48|CHI2E34|475.0| 172.89999999999964|
| BR-040|                48|CHI4A56|835.0|  892.7199999999998|
| BR-040|                48|CHI6Y78|565.0|            -112.88|
| BR-040|                48|COL7A78|475.0|  50.15999999999989|
| BR-040|                48|COL9Q01|655.0|             1025.3|
| BR-040|                48|PAR5N67|385.0|  31.60000000000008|
| BR-040|                48|PER5A56|565.0|             -84.88|
| BR-040|                48|VEN4M56|475.0| -75.47999999999996|
| BR-040|               154|ARG3R45|655.0|-223.50000000

In [39]:
from pyspark.sql import Window
from pyspark.sql.functions import lag, col

df_cruzamento = ss.createDataFrame([], "rodovia: string, placa: string, tempo_inicio: bigint, tempo_final: bigint")

Velocidades_Maximas = p.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
Aceleracoes_Maximas = p.select(F.col('rodovia'), 0.8*F.col("AceleracaoMaxima"))
Aceleracoes_Maximas = Aceleracoes_Maximas.withColumnRenamed("(AceleracaoMaxima * 0.8)", "AceleracaoMaxima")

collision_tolerance = 1
collision_tolerance_quad = 0.5
def processa_carro(df, df_cruzamento):
    windowSpec = Window.partitionBy("placa").orderBy("tempo_da_simulacao")
    df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
    df = df.withColumn("prev_tempo_da_simulacao", lag("tempo_da_simulacao", 1).over(windowSpec))
    df = df.withColumn("vel_y", (col("pos_y") - col("prev_pos_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))
    df = df.withColumn("prev_vel_y", lag("vel_y", 1).over(windowSpec))
    df = df.withColumn("acel_y", (col("vel_y") - col("prev_vel_y")) / (col("tempo_da_simulacao") - col("prev_tempo_da_simulacao")))
    
    processa_velocidade_media(df)
    
    df = df.withColumn("posicao_prevista", col("pos_y") + col("vel_y") * (collision_tolerance) + col("acel_y") * collision_tolerance_quad)
    
    window_spec_rf = Window.partitionBy("rodovia", "pos_x").orderBy('pos_y')
    lag_column = col("posicao_prevista") - lag(col("posicao_prevista")).over(window_spec_rf)
    lead_column = lead(col("posicao_prevista")).over(window_spec_rf) - col("posicao_prevista")
    
    # Add the lag column to the DataFrame
    df = df.withColumn("Risco_Colisão", when(((lag_column < 0) | (lead_column < 0)), 1).otherwise(0))
    
    df = df.join(Velocidades_Maximas,on='rodovia',how='left')
    df = df.join(Aceleracoes_Maximas,on='rodovia',how='left')
    
    df = df.withColumn('acima_vel',F.abs(col('vel_y'))>F.abs(col('VelocidadeMaxima')))
    df = df.withColumn('acima_acel',F.abs(col('acel_y'))>F.abs(col('AceleracaoMaxima')))
    
    #Calcula acima_vel
    
    df = df.withColumn("troca_faixa", col("pos_x") != lag("pos_x", 1).over(windowSpec))
    
    #Contador de trocas
    
    df = df.withColumn('multado',((F.col('acima_vel') == 1) & (lag('acima_vel').over(windowSpec) == 0)))
    
    #Multa
    
    windowSpec = Window.partitionBy('rodovia',"placa").orderBy('tempo_da_simulacao')
    
    df = df.withColumn("prev_pos_y", lag("pos_y", 1).over(windowSpec))
    df = df.withColumn("on_road", (((col("pos_y") > 0) & (col('pos_y') < 800))))
    
    df = df.withColumn('tempo_inicio',when(((F.col('on_road') == True) & (lag('on_road').over(windowSpec) == False)), F.col("tempo_da_simulacao")).otherwise(None))
    df = df.withColumn('tempo_final',when(((F.col('on_road') == True) & (lead('on_road').over(windowSpec) == False)), F.col("tempo_da_simulacao")).otherwise(None))
    df_cruzamento_aux = df.select('rodovia', 'placa', 'tempo_inicio','tempo_final').filter((F.col('vel_y') != 0) & (F.col('tempo_inicio').isNotNull() | F.col('tempo_final').isNotNull()))
    
    df_cruzamento = df_cruzamento.union(df_cruzamento_aux)
    
    windowSpec2 = Window.partitionBy('placa','rodovia').orderBy('tempo_final')
    
    df_cruzamento = df_cruzamento.withColumn('tempo_cruzamento', col('tempo_final') - lag('tempo_inicio').over(windowSpec2))
    
    processa_tempo_cruzamento(df_cruzamento)
    df_cruzamento = df_cruzamento.filter(df_cruzamento.tempo_cruzamento.isNull())


    print(df_cruzamento.filter(col("tempo_final").isNotNull()).toPandas().sort_values(by='tempo_inicio').head(20))
processa_carro(df, df_cruzamento)
pipeline(df, df_cruzamento)

KeyError: 'BR-040'

In [26]:
pd.set_option('display.max_rows', 100)
ata = df.orderBy(df.tempo_da_simulacao.desc()).toPandas()
ata

,rodovia,tempo_da_simulacao,placa,pos_x,pos_y,prev_pos_y,prev_tempo_da_simulacao,vel_y,prev_vel_y,acel_y,...,Risco_Colisão,VelocidadeMaxima,AceleracaoMaxima,acima_vel,acima_acel,troca_faixa,multado,on_road,tempo_inicio,tempo_final
0,BR-040,715,ARG7P89,835.0,-79.88,-77.88,714.0,-2.00,-2.00,0.00,...,1,10,0.096,False,False,False,False,False,NaN,NaN
1,BR-040,715,ECU8D89,655.0,476.20,478.20,714.0,-2.00,-2.00,0.00,...,0,10,0.096,False,False,False,False,True,NaN,NaN
2,BR-040,715,GUY1X23,385.0,-53.06,-57.58,714.0,4.52,4.40,0.12,...,0,10,0.096,False,True,False,False,False,NaN,NaN
3,BR-040,715,PAR3Z45,655.0,268.42,270.42,714.0,-2.00,-2.00,0.00,...,0,10,0.096,False,False,False,False,True,NaN,NaN
4,BR-040,715,URU2O34,745.0,942.06,942.06,714.0,0.00,0.00,0.00,...,1,10,0.096,False,False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11116,BR-040,7,PAR5N67,385.0,-148.00,NaN,NaN,NaN,NaN,NaN,...,0,10,0.096,None,None,None,None,False,NaN,NaN
11117,BR-040,6,ARG3R45,655.0,1030.06,1040.02,5.0,-9.96,-9.98,0.02,...,1,10,0.096,False,False,False,False,False,NaN,NaN
11118,BR-040,5,ARG3R45,655.0,1040.02,1050.00,4.0,-9.98,-10.00,0.02,...,1,10,0.096,False,False,False,False,False,NaN,NaN
11119,BR-040,4,ARG3R45,655.0,1050.00,1060.00,3.0,-10.00,NaN,NaN,...,0,10,0.096,False,None,False,False,False,NaN,NaN


In [27]:
df_cruzamento.orderBy(df_cruzamento.placa.asc()).show(1000)

+-------+------------+-----------+--------------------+-------+-------+-------------------+----+
|on_road|tempo_inicio|tempo_final|               pos_y|  placa|rodovia|              vel_y|diff|
+-------+------------+-----------+--------------------+-------+-------+-------------------+----+
|   true|         220|       null|   7.660000000000025|ARG1L23| BR-040|  7.739999999999995|null|
|   true|        null|        331|              790.06|ARG1L23| BR-040|  6.039999999999964| 111|
|   true|          30|       null|   797.0199999999998|ARG3R45| BR-040| -9.480000000000018|null|
|   true|        null|        123|  2.7999999999968423|ARG3R45| BR-040| -7.620000000000047|  93|
|   true|         275|       null|   798.1199999999999|ARG7P89| BR-040|               -2.0|null|
|   true|        null|        675| 0.11999999999989086|ARG7P89| BR-040|               -2.0| 400|
|   true|         186|       null|   798.3999999999999|ARG9D01| BR-040|               -2.0|null|
|   true|         218|       n

In [28]:
tempo_medio

{'BR-040': 0,
 'BR-116': 0,
 'BR-135': 0,
 'BR-393': 0,
 'BR-101': 0,
 'BR-376': 0,
 'BR-262': 0,
 'BR-153': 0,
 'BR-230': 0,
 'BR-349': 0,
 'BR-060': 0,
 'BR-050': 0,
 'BR-070': 0,
 'BR-163': 0,
 'BR-277': 0}

In [14]:
vel_media

{'BR-040': 2.043637465629274,
 'BR-116': 0,
 'BR-135': 0,
 'BR-393': 0,
 'BR-101': 0,
 'BR-376': 0,
 'BR-262': 0,
 'BR-153': 0,
 'BR-230': 0,
 'BR-349': 0,
 'BR-060': 0,
 'BR-050': 0,
 'BR-070': 0,
 'BR-163': 0,
 'BR-277': 0}

In [ ]:
dfp = df.toPandas()
dfp[dfp['placa'] == 'PAR7P89'].sort_values(by=['tempo_da_simulacao'], ascending=True)